Last updated: 16 Feb 2023

# 👋 PyCaret Time Series Forecasting Tutorial

PyCaret is an open-source, low-code machine learning library in Python that automates machine learning workflows. It is an end-to-end machine learning and model management tool that exponentially speeds up the experiment cycle and makes you more productive.

Compared with the other open-source machine learning libraries, PyCaret is an alternate low-code library that can be used to replace hundreds of lines of code with a few lines only. This makes experiments exponentially fast and efficient. PyCaret is essentially a Python wrapper around several machine learning libraries and frameworks, such as scikit-learn, XGBoost, LightGBM, CatBoost, spaCy, Optuna, Hyperopt, Ray, and a few more.

The design and simplicity of PyCaret are inspired by the emerging role of citizen data scientists, a term first used by Gartner. Citizen Data Scientists are power users who can perform both simple and moderately sophisticated analytical tasks that would previously have required more technical expertise.


# 💻 Installation

PyCaret is tested and supported on the following 64-bit systems:
- Python 3.7 – 3.10
- Python 3.9 for Ubuntu only
- Ubuntu 16.04 or later
- Windows 7 or later

You can install PyCaret with Python's pip package manager:

`pip install pycaret`

PyCaret's default installation will not install all the extra dependencies automatically. For that you will have to install the full version:

`pip install pycaret[full]`

or depending on your use-case you may install one of the following variant:

- `pip install pycaret[analysis]`
- `pip install pycaret[models]`
- `pip install pycaret[tuner]`
- `pip install pycaret[mlops]`
- `pip install pycaret[parallel]`
- `pip install pycaret[test]`

In [ ]:
# install pycaret
#!pip install pycaret[full]
# check installed version
import pycaret 
pycaret.__version__

# 🚀 Quick start

PyCaret's time series forecasting module is now available. The module currently is suitable for univariate / multivariate time series forecasting tasks. The API of time series module is consistent with other modules of PyCaret.

It comes built-in with preprocessing capabilities and over 30 algorithms comprising of statistical / time-series methods as well as machine learning based models. In addition to the model training, this module has lot of other capabilities such as automated hyperparameter tuning, ensembling, model analysis, model packaging and deployment capabilities. 

A typical workflow in PyCaret consist of following 5 steps in this order:

### **Setup** ➡️ **Compare Models** ➡️ **Analyze Model** ➡️ **Prediction** ➡️ **Save Model** <br/>

In [ ]:
# import pandas module
import pandas as pd
import re
import numpy as np

# global variables
Soil_humi = []
Resistance = []
Soil_temp = []
total_amount = 0
rolling_mean_window_data = 15
rolling_mean_window_grouped = 5
min_length_df = 300 #was 50
slope_irrigation = -0.07

# creating a data frame
data = pd.read_csv("binned_removed_new.csv",header=0)
data.head()

# create array with sensors strings
for col in data.columns:
    if re.findall(r"SoilHumiditySensor$", col):
        Soil_humi.append(col)
    elif re.findall(r"\/ $", col):
        Resistance.append(col)
    elif re.findall(r"SoilThermometer$", col):
        Soil_temp.append(col)

# print them to check
for i in range(len(Soil_humi)):
    total_amount += 1
    print(Soil_humi[i])
for i in range(len(Resistance)):
    total_amount += 1
    print(Resistance[i])
for i in range(len(Soil_temp)):
    total_amount += 1
    print(Soil_temp[i])
    
print("Total amount of values: ", total_amount)

In [ ]:
# plot the dataset
data.plot(figsize = (20,10))

In [ ]:
# Show if there are any missing values inside the data
data.isna().any()

In [ ]:
data['Time'] = data['Time'].str[:-9]
data['Time']=pd.to_datetime(data['Time'])
data = data.set_index('Time')

data.plot(subplots=True, figsize=(20,50))

In [ ]:
data = data.fillna(method = 'bfill')
data.plot(subplots=True, figsize=(20,50))

In [ ]:
# creating a data frame
data_weather = pd.read_csv("april_weather.csv", header=0, sep=";")
data_weather.head()

In [ ]:
# Convert the datetime column to a datetime object
data_weather['Heure'] = pd.to_datetime(data_weather['Heure'], format=' %d/%m/%Y %H:%M:%S')#, errors='coerce')
#print("After datetime, before ISO:  \n", data_weather, "\n", data_weather.dtypes, "\n")

# Format the datetime object as an ISO time string
data_weather['Heure'] = data_weather['Heure'].dt.strftime('%Y-%m-%dT%H:%M:%S')

#change name
data_weather = data_weather.rename(columns={'Heure':'Time'})

# change datatype
data_weather['Time'] = pd.to_datetime(data_weather['Time'])

# set the index of the DataFrame to the timestamp column
data_weather = data_weather.set_index('Time')

In [ ]:
# Drop useless
data_weather = data_weather.drop(['Température intérieure(°C)', 'Humidité intérieure(%)', 'Intervalle', 'NO.'], axis=1)

In [ ]:
data_weather.head()

In [ ]:
# Convert wind direction to integer
directions = [" N", " NNE", " NE", " ENE", " E", " ESE", " SE", " SSE", " S", " SSW", " SW", " WSW", " W", " WNW", " NW", " NNW"]
target_col = 'Direction du vent'

for index, value in data_weather[target_col].iteritems():
    # Calculate wind direction in degrees -> TODO : convert to int and round
    if value == " ---":
        data_weather[target_col][index] = np.nan
    else:
        data_weather[target_col][index] = directions.index(value) * 22.5
        
data_weather[target_col] = data_weather[target_col].astype("float64")

In [ ]:
# convert to float64
ref_dtype = 'float64'
obj_dtype = 'object'

for col in data_weather.columns:
    if data_weather[col].dtype == obj_dtype:
        data_weather[col] = pd.to_numeric(data_weather[col], errors='coerce')
        
    print("This is :",col, "it has the following dtype: ", data_weather[col].dtype)

In [ ]:
data_weather.head()

In [ ]:
# preserve old datetimeindex
timestamps = data_weather.index
data_weather.reset_index(drop=True)

In [ ]:
# TODO: compare them with orig data
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import BayesianRidge, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

# Init the transformer
reg_imp = IterativeImputer(estimator=Ridge())

# Fit/transform
data_weather = pd.DataFrame(reg_imp.fit_transform(data_weather), columns=data_weather.columns)

# restore old index
data_weather['Time'] = timestamps
data_weather = data_weather.set_index('Time')
data_weather.index = pd.to_datetime(data_weather.index)

In [ ]:
#ref_dtype = 'float64'
# iterate through the columns and compare their data types to the reference data type
#for col in data_weather.columns:
#    print("This is :",col)
#    if data_weather[col].dtype == 'datetime64[ns]':
#        data_weather[col] = data_weather[col].resample('10T').ffill()
#    if data_weather[col].dtype == ref_dtype:
#        data_weather[col] = data_weather[col].resample('10T').mean()
#    else:
#        data_weather[col] = data_weather[col].resample('10T').ffill()

In [ ]:
data_weather.head()

In [ ]:
data_weather_re = data_weather.resample('10T').fillna(method = 'bfill') #TODO: fill with mean!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
data_weather_re.index

In [ ]:
data_weather_re.head(10)

In [ ]:
# concatenate the DataFrames vertically
#data_comb = pd.concat([data, data_weather_re], axis=0)
data_comb = pd.merge(data, data_weather_re, left_index=True, right_index=True, how='outer')

# print the resulting DataFrame
data_comb.tail(10)

In [ ]:
import missingno as msno 
# Plot correlation heatmap of missingness
msno.matrix(data_comb)

In [ ]:
# Select the rows between two dates => cut dataframe
start_date = str(data_weather_re.index[0])
end_date = str(data_weather_re.index[-1])
data_comb_cut = data_comb.loc[start_date:end_date]

data = data_comb_cut

In [ ]:
msno.matrix(data)

In [ ]:
data.head()

In [ ]:
data.dtypes

In [ ]:
ref_dtype = 'float64'
obj_dtype = 'object'

for col in data.columns:
    if data[col].dtype == obj_dtype:
        data[col] = pd.to_numeric(data[col], errors='coerce')
    #if data[col].dtype == ref_dtype:
        #data[col] = data[col].rolling(window=rolling_mean_window_data, win_type='gaussian').mean(std=rolling_mean_window_data)
        
    print("This is :",col, "it has the following dtype: ", data[col].dtype)

In [ ]:
msno.heatmap(data)

In [ ]:
msno.dendrogram(data)

In [ ]:
data.isna().any()

In [ ]:
data.plot(subplots=True, figsize=(20,50))

We can use the following estimators:
-----------------------------------
LinearRegression: LinearRegression

BayesianRidge: Regularized linear regression.

DecisionTreeRegressor: Decision tree regression. => no

ExtraTreesRegressor: Extremely randomized trees regression. => slow

KNeighborsRegressor: k-Nearest Neighbors regression. => no

RandomForestRegressor: Random forest regression. => slow

Ridge: Linear least squares with l2 regularization. => best?

SVR: Support vector regression. => slow, no

In [ ]:
# TODO: compare them with orig data

# Fit/transform
#for col in data.columns:
#    # Init the transformer
#    reg_imp = IterativeImputer(estimator=SVR())
#    # Fit/transform
#    data_imp[col] = pd.DataFrame(reg_imp.fit_transform(data[[col]]), columns=[col])

In [ ]:
# Check for missing values
print(data.isna().any())


In [ ]:
#from sklearn.impute import KNNImputer

#for col in data.columns:
#    if data[col].dtype == ref_dtype:
#        #data_imp[col] = data[col].rolling(window=rolling_mean_window_data, win_type='gaussian').mean(std=rolling_mean_window_data)
#        # Init the imputer
#        imputer = KNNImputer(n_neighbors=rolling_mean_window_data)
#
#        # Impute the missing values in column col
#        data[col] = imputer.fit_transform(data[[col]])

In [ ]:
data.plot(subplots=True, figsize=(20,50))

In [ ]:
data.head()

In [ ]:
def compare_dataframes(df1, df2):
    # Verify column names
    if not set(df1.columns) == set(df2.columns):
        print("Column names are not the same in both dataframes.")
        return

    # Find the common columns between the two dataframes
    common_columns = df1.columns.intersection(df2.columns)

    # Compare the available values in the common columns
    for column in common_columns:
        values1 = df1[column].dropna()
        values2 = df2[column].dropna()

        min_length = min(len(values1), len(values2))
        values1 = values1[:min_length]
        values2 = values2[:min_length]

        diff = np.abs(values1.values - values2.values)
        mae = np.mean(diff)
        rmse = np.sqrt(np.mean(diff ** 2))
        mpe = np.mean(diff / values1.values) * 100

        print(f"Column: {column}")
        print(f"MAE: {mae:.2f}")
        print(f"RMSE: {rmse:.2f}")
        print(f"MPE: {mpe:.2f}%")
        print("df1 len:",len(values1),"df2 len:",len(values2))
        print()

#compare_dataframes(data, data_imp)

In [ ]:
#data_imp.index = data.index
#data = data_imp

In [ ]:
#data_imp.plot(subplots=True, figsize=(20,50))

In [ ]:
#impute data
#from sklearn.impute import KNNImputer

##data_knn_imputed = data_knn_imputed.drop(['Time'], axis=1)

# Init the transformer
#knn_imp = KNNImputer(n_neighbors=10)

# Fit/transform
#data_imp.iloc[:,0:len(data)] = knn_imp.fit_transform(data_imp.iloc[:,0:len(data)])

In [ ]:
#data_imp.plot(subplots=True, figsize=(20,50))

In [ ]:
# Plot correlation heatmap of missingness
msno.matrix(data)

In [ ]:
# Show if there are any missing values inside the data
data.isna().any()

In [ ]:
# create average cols
data['grouped_soil'] = data[Soil_humi].mean(axis=1)
data['grouped_resistance'] = data[Resistance].mean(axis=1)
data['grouped_soil_temp'] = data[Soil_temp].mean(axis=1)

In [ ]:
data.index.dtype

In [ ]:
#data['Time'] = data['Time'].str[:-9]
#data.head()

In [ ]:
#data['Time']=pd.to_datetime(data['Time'])

#data.head()

In [ ]:
# after concat => need to convert to datetimeindex once more:
#data.index=pd.to_datetime(data.index, format='%Y-%m-%d %H:%M:%S') #2023-04-01 00:50:00

In [ ]:
## Create Features
## create rolling mean: introduces NaN again -> later just cut off
data['rolling_mean_grouped_soil'] = data['grouped_soil'].rolling(window=rolling_mean_window_grouped, win_type='gaussian').mean(std=rolling_mean_window_grouped)
data['rolling_mean_grouped_soil_temp'] = data['grouped_soil_temp'].rolling(window=rolling_mean_window_grouped, win_type='gaussian').mean(std=rolling_mean_window_grouped)

data['hour'] = data.index.hour#.astype("float64")
data['minute'] = data.index.minute#.astype("float64")
data['date'] = data.index.day#.astype("float64")
data['month'] = data.index.month#.astype("float64")
data['day_of_year'] = data.index.dayofyear#.astype("float64")

In [ ]:
# feature scaling
data.dtypes

In [ ]:
# Min-Max Normalization
def max_min_norm(df_all):
    df = df_all.drop(['rolling_mean_grouped_soil', 'hour', 'minute', 'date', 'month', 'day_of_year'], axis=1)
    index = df.index
    df_norm = pd.DataFrame()
    for col in df.columns:
        print("Current col: ", col)
        if df[col].dtype == ref_dtype:
            print("Normalize:   ", col)
            df_norm[col] = (df[col]-df[col].min())/(df[col].max()-df[col].min())
    df_norm = pd.concat([df_norm, df_all['hour'], df_all['minute'], df_all['date'], df_all['month'], df_all['day_of_year'], df_all.rolling_mean_grouped_soil], 1)
    return df_norm

#max_min_norm(data)

In [ ]:
data.dtypes

In [ ]:
print(str(Soil_temp).replace("[", "").replace("]", ""))

In [ ]:
int(5/2 + 0.5)

In [ ]:
data.head()

In [ ]:
#data = data[['Time', 'hour', 'minute', 'date', 'month', 'grouped_soil', 'grouped_resistance', 'grouped_soil_temp', 'rolling_mean_grouped_soil', 'rolling_mean_grouped_soil_temp', 
#             str(Soil_humi).replace("[", "").replace("]", "").replace("\\",""),
#             str(Resistance).replace("[", "").replace("]", "").replace("\\",""),
#             str(Soil_temp).replace("[", "").replace("]", "").replace("\\","")
#            ]]

# drop those values without rolling_mean
#data = data[rolling_mean_window_data+int(rolling_mean_window_grouped/2 + 0.5):]
data = data[4:]

# reset changed index(due to drop)
#data = data.reset_index()

#data = data.drop(['index'],axis=1)

In [ ]:
print(data.iloc[0])

data.head(10)

In [ ]:
#data_plot = data.drop(['hour','minute','date','month','grouped_soil','grouped_resistance','grouped_soil_temp', Soil_humi, Resistance, Soil_temp], axis=1)
#data_plot.tail(20)

In [ ]:
#data_plot.set_index('Time').plot(figsize = (20,10))

In [ ]:
# Split dataset into train and test set
def split_data(data, split_date):
    return data[data.index <= split_date].copy(), \
           data[data.index >  split_date].copy()

train, test = split_data(data, pd.to_datetime('2023-04-25 15:30')) # splitting the data for training before 15th June

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))
plt.xlabel('Time')
plt.ylabel('rolling_mean_grouped_soil')
plt.plot(train.index,train['rolling_mean_grouped_soil'],label='train')
plt.plot(test.index,test['rolling_mean_grouped_soil'],label='test')
plt.legend()
plt.show()

In [ ]:
import numpy as np
# calculate slope of "rolling_mean_grouped_soil"
f = data.rolling_mean_grouped_soil
data['gradient'] = np.gradient(f)
#data.iloc[3270:3330]
data.head(10)

In [ ]:
# create dataframe with downward slope
neg_slope = pd.DataFrame({"Time":[],
                          "index":[],
                          "rolling_mean_grouped_soil":[],
                          "gradient":[]
                         })
i=0
for index, row in data.iterrows():
    if row['gradient'] < slope_irrigation:
        print(index, row['rolling_mean_grouped_soil'], row['gradient'])
        current_series = pd.Series([index, int(i), row['rolling_mean_grouped_soil'], row['gradient']], index=['Time', 'index', 'rolling_mean_grouped_soil', 'gradient'])#.to_frame().T
        neg_slope = neg_slope.append(current_series,ignore_index=True)
    i += 1
len(neg_slope)

In [ ]:
type(neg_slope)

In [ ]:
#neg_slope = neg_slope.reset_index(inplace=True)
neg_slope.head(50)

In [ ]:
# dont ask, I love pandas^
#neg_slope_2 = pd.DataFrame({'Time':[], 'rolling_mean_grouped_soil':[], 'gradient': []})
#neg_slope_2 = pd.concat([neg_slope_2, neg_slope], ignore_index=True)
#neg_slope_2.head()

In [ ]:
#neg_slope_2 = neg_slope_2.reset_index(inplace=True)
#neg_slope = neg_slope_2
#neg_slope.head()

In [ ]:
#neg_slope = neg_slope.reset_index()
#neg_slope.drop(index, axis=1).head()

In [ ]:
def delete_nonconsecutive_rows(df, column_name, min_consecutive):
    arr = df['index'].to_numpy()
    i = 0
    while i < len(arr) - 1:
        if arr[i+1] == arr[i] + 1:
            start_index = i
            while i < len(arr) - 1 and arr[i+1] == arr[i] + 1:
                i += 1
            end_index = i
            if end_index - start_index + 1 < min_consecutive:
                df = df.drop(range(start_index, end_index+1))
        i += 1
    return df

neg_slope = delete_nonconsecutive_rows(neg_slope, 'index', 5)
print(neg_slope)

In [ ]:
# visualize areas with downward slope
ax = data.drop(['hour','minute','date','month','grouped_soil','grouped_resistance','grouped_soil_temp','day_of_year'], axis=1).plot(figsize = (20,10))

In [ ]:
# Create visual representation of irrigation times
def highlight(data_plot,ax):
    for index, value in neg_slope['index'].iteritems():
        print(value)
        ax.axvspan(value-10, value+10, facecolor='pink', edgecolor='none', alpha=.5)
        
highlight(data, ax)

ax.figure.suptitle("""Irrigation times highlighted\n\n""", fontweight ="bold") 
#ax.figure(figsize=(20,10))
ax.figure.savefig('irrigation_times.png', dpi=400)

In [ ]:
neg_slope_indices = neg_slope['index'].to_numpy()
print(neg_slope_indices)

In [ ]:
#print(neg_slope_indices.astype(np.int32))
#print(type(neg_slope_indices[0]))

In [ ]:
# Convert each element to an integer
# Creating a NumPy ndarray with a specific datatype
#neg_slope_int_indices = np.array([], dtype=np.int32)
#print(type(neg_slope_int_indices))

#for i in range(len(neg_slope_indices)):
#    np.append(neg_slope_int_indices,int(neg_slope_indices[i]),axis=None)

#print(neg_slope_indices)

In [ ]:
#print(type(neg_slope_indices[0]))
#print(neg_slope_indices)

In [ ]:
def create_split_tuples(df, indices_to_omit):
    # Sort the indices in ascending order
    indices_to_omit = sorted(indices_to_omit)

    # Create a list of index ranges to remove
    ranges_to_remove = []
    start_idx = None
    for idx in indices_to_omit:
        if start_idx is None:
            start_idx = idx
        elif idx == start_idx + 1:
            start_idx = idx
        else:
            ranges_to_remove.append((int(start_idx), int(idx-1)))
            start_idx = idx
    if start_idx is not None:
        ranges_to_remove.append((int(start_idx), len(df)))
        
    print("Irrigation times to be omitted: ", ranges_to_remove)
    print("type: ", type(ranges_to_remove[0][0]))

    return ranges_to_remove

tuples_to_remove = create_split_tuples(data, neg_slope_indices)
print(tuples_to_remove)

In [ ]:
data['index'] = data.reset_index().index

In [ ]:
def split_dataframe(df, index_ranges):
    dfs = []
    for i, (start, end) in enumerate(index_ranges):
        print()
        if index_ranges[i][1]-index_ranges[i][0] < min_length_df:
            continue
        else:
            dfs.append(df.iloc[index_ranges[i][0]:index_ranges[i][1]])
    return dfs

sub_dfs = split_dataframe(data, tuples_to_remove) 

In [ ]:
sub_dfs[0].head(2)

In [ ]:
# print dataframes
print("There are ", len(sub_dfs), " dataframes now.")
for sub_df in sub_dfs:
    print(sub_df.iloc[0:1])
    print(len(sub_df))
    sub_df.drop(['index', 'grouped_soil', 'grouped_resistance', 'grouped_soil_temp', 'hour', 'minute', 'date', 'month', 'gradient', 'day_of_year'],axis=1).plot(figsize = (20,10))

In [ ]:
# find global max and min in all sub_dfs and cut them from min to max 
# => train data will start with min and end with max 
cut_sub_dfs = []
for i in range(len(sub_dfs)):
    # reset "new" index
    sub_dfs[i] = sub_dfs[i].reset_index()
    
    # index
    global_min_index = sub_dfs[i]['rolling_mean_grouped_soil'].idxmin()
    global_max_index = sub_dfs[i]['rolling_mean_grouped_soil'].idxmax()
    # value
    global_min = sub_dfs[i]['rolling_mean_grouped_soil'].min()
    global_max = sub_dfs[i]['rolling_mean_grouped_soil'].max()
    
    if global_max_index-global_min_index > 0:
        print(i,": ",global_min_index, "value:", global_min, global_max_index, "value:", global_max, "length:", global_max_index-global_min_index)
        cut_sub_dfs.append(sub_dfs[i].iloc[global_min_index:global_max_index])
#sub_dfs[1].drop(['Time'],axis=1).plot()

In [ ]:
for df in cut_sub_dfs:
    print(df.index)
    #df.drop(['index', 'hour', 'minute', 'date', 'month', 'day_of_year'],axis=1).plot()

In [ ]:
#from scipy.signal import argrelextrema
#
#for df in cut_sub_dfs:
#    # Find local maxima and minima
#    df['min'] = df.iloc[argrelextrema(df['rolling_mean_grouped_soil'].values, np.less_equal, order=2)[0]]['rolling_mean_grouped_soil']
#    df['max'] = df.iloc[argrelextrema(df['rolling_mean_grouped_soil'].values, np.greater_equal, order=2)[0]]['rolling_mean_grouped_soil']
#
#    # Plot data and local extrema
#    plt.plot(df['rolling_mean_grouped_soil'])
#    plt.scatter(df.index, df['min'], c='r')
#    plt.scatter(df.index, df['max'], c='g')
#    plt.show()

In [ ]:
cut_sub_dfs[0].head()

In [ ]:
for i in range(len(cut_sub_dfs)):
    # clean dataframe
    #cut_sub_dfs[i] = cut_sub_dfs[i].reset_index()
    cut_sub_dfs[i] = cut_sub_dfs[i].set_index('Time')
    cut_sub_dfs[i] = cut_sub_dfs[i].rename(columns={'index':'orig_index'})

In [ ]:
cut_sub_dfs[0].head()

In [ ]:
i = 1
for df in cut_sub_dfs:
    print("Dataframe: ",i)
    i+=1
    print(df.iloc[:1])
    #df.drop(['orig_index', 'Time', 'hour', 'minute', 'date', 'month', 'gradient'],axis=1).plot()

In [ ]:
# save all dataframes to one and rename
df_comb = pd.DataFrame()
for i in range(len(cut_sub_dfs)):
    # copy elements to one df_comb
    
    df_comb = pd.concat([df_comb, cut_sub_dfs[i]['orig_index']], axis=1)
    df_comb = pd.concat([df_comb, cut_sub_dfs[i]['rolling_mean_grouped_soil']], axis=1)
    
    df_comb = df_comb.rename(columns={'orig_index':'orig_index_' + str(i)})
    df_comb = df_comb.rename(columns={'rolling_mean_grouped_soil':'rolling_mean_grouped_soil_' + str(i)})
    
df_comb.head(25)

In [ ]:
# series are not of same length => visualized here!
import re

df_comb_plot = df_comb
pattern = r"^orig_index_*"

for colname in df_comb.columns:
    if re.search(pattern, colname):
        print("Drop: ", colname)
        df_comb_plot = df_comb_plot.drop(columns=colname, axis=1)
        
df_comb_plot.plot(figsize = (20,10))
print(len(df_comb_plot.columns))

## Create lag features 
In this section lag features are created to increase the models performance with former soil drying examples.

### TODO: Omit lag features that are shorter than current dataframe or series.

In [ ]:
j=1
for cut_sub_df in cut_sub_dfs:
    print("Start time of df",j,":",cut_sub_df.index[0].time(), "Length: ",len(cut_sub_df))
    j+=1
print("Should be aligned with:",cut_sub_dfs[len(cut_sub_dfs)-1].index[0].time())

In [ ]:
# Always take the last period and enhance with formerly recorded data
def create_lags():
    lag_periods = len(cut_sub_dfs)
    # copy last df initially to df_lag
    df_lag = cut_sub_dfs[lag_periods-1]
    timestamp_to_align = cut_sub_dfs[lag_periods-1].index[0].time()

    print("Timestamp to align older dfs with: ",timestamp_to_align)
    
    # cut previous recorded dfs according to timestamp_to_align
    j = 0
    for cut_sub_df in cut_sub_dfs[:-1]:
        i = 0
        for index, row in cut_sub_df.iterrows():
            #print(i,index.time())
            if (timestamp_to_align == index.time()):
                print("Found match:",index.time(), "Present in dataframe: ",j,"At the index:",i)
                # drop rows in current dataframe that are before
                cut_sub_dfs[j] = cut_sub_df.iloc[i:]
                print("Should be the same like:",cut_sub_dfs[j].index[0].time())
                break
            i+=1
        
        # rename and append previous soil drying examples to df_lag
        for col in cut_sub_dfs[j].columns:
            new_col_name = col + "_lag_" + str(j + 1)
            append_col = cut_sub_dfs[j].rename(columns={col: new_col_name})
            append_col = append_col.reset_index(drop=True)
            #print(append_col)
            
            # append it to df_lag:
            #df_lag = df_lag.join(append_col[new_col_name])#.reset_index(drop=True))    #n investigate merge()
            #df_lag = pd.concat([df_lag, append_col[new_col_name]], axis=1)
            #df_lag = pd.concat([df_lag, append_col[new_col_name]], axis = 1)
            
            # Concatenate the smaller dataframe with the larger dataframe horizontally
            #merged_df = pd.concat([small_df, large_df], axis=1)

            # Alternatively, you can use merge to join the dataframes on a common column
            merged_df = df_lag.merge(append_col[new_col_name], left_index=True, right_on='numerical_index_column')

            #print("This is one:",append_col[new_col_name])
            
        j+=1
    
    return df_lag

# Call create_lags
df_lag = create_lags()

#print(df_lag.dtypes)

# Plot correlation heatmap of missingness
msno.matrix(df_lag)
df_lag.to_csv('df_lag_my.csv', index=True)

In [ ]:
# always take the last period and enhance with formerly recorded data
def create_lags():
    df_lag = pd.DataFrame()
    lag_periods = len(cut_sub_dfs)
    
    for cut_sub_df in cut_sub_dfs:
        # an index does not have access to the dt object -> copy index to new column
        cut_sub_df["TIME_COPY"] = cut_sub_df.index
        cut_sub_df["TIME_COPY"] = pd.to_datetime(cut_sub_df["TIME_COPY"])

    last_df = cut_sub_dfs[lag_periods - 1]
    df_lag = last_df.copy()

    for index, cut_sub_df in enumerate(cut_sub_dfs[:lag_periods - 1]):
        aligned_df = pd.DataFrame(index=last_df.index)
        aligned_df = aligned_df.reindex(last_df.index, method='nearest')

        for column in cut_sub_df.columns:
            aligned_df[column] = cut_sub_df[column].reindex(last_df.index, method='nearest')

        df_lag = pd.concat([df_lag, aligned_df], axis=1)

    return df_lag

# call create_lags
df_lag = create_lags()

# Plot correlation heatmap of missingness
msno.matrix(df_lag)
print(df_lag.iloc[0:3])
df_lag.to_csv('df_lag.csv', index=True)

In [ ]:
# always take the last period and enhance with formerly recorded data
def create_lags():
    df_lag = pd.DataFrame()
    lag_periods = len(cut_sub_dfs)
    first_timestamp = []
    time_differences = []

    # iterate through dfs to get to know first timestamps of individual series
    for cut_sub_df in cut_sub_dfs:
        #append timestamps to array 
        first_timestamp.append(cut_sub_df.index[0])
    print(first_timestamp)

    for index, value in enumerate(first_timestamps):
        
    return df_lag
    
df_lag = create_lags()


## Setup
This function initializes the training environment and creates the transformation pipeline. Setup function must be called before executing any other function in PyCaret. `Setup` has only one required parameter i.e. `data`. All the other parameters are optional.

In [ ]:
cut_sub_dfs[2].head()

Once the setup has been successfully executed it shows the information grid containing experiment level information. 

- **Session id:**  A pseudo-random number distributed as a seed in all functions for later reproducibility. If no `session_id` is passed, a random number is automatically generated that is distributed to all functions.<br/>
<br/>
- **Approach:**  Univariate or multivariate. <br/>
<br/>
- **Exogenous Variables:**  Exogeneous variables to be used in model. <br/>
<br/>
- **Original data shape:**  Shape of the original data prior to any transformations. <br/>
<br/>
- **Transformed train set shape :**  Shape of transformed train set <br/>
<br/>
- **Transformed test set shape :**  Shape of transformed test set <br/>
<br/>

PyCaret has two set of API's that you can work with. (1) Functional (as seen above) and (2) Object Oriented API.

With Object Oriented API instead of executing functions directly you will import a class and execute methods of class.

In [ ]:
# Create a Lasso model with alpha=1.0 and a Ridge model with alpha=1.0
#lasso_model = create_model('lasso', alpha=1.0, data=data, session_id=1235)
#ridge_model = create_model('ridge', alpha=1.0, data=data, session_id=1234)

# Compare all available classification models, including the Lasso and Ridge models
#models = compare_models(models=[lasso_model, ridge_model], data=train_data, sort='Accuracy', n_select=3, fold=5, round=4)

In [ ]:
# import TSForecastingExperiment and init the class
#from pycaret.regression import *
from pycaret.time_series import *
exp=[]

for i in range(len(cut_sub_dfs)):
    exp.append(TSForecastingExperiment())
    
    # check the type of exp
    type(exp[i])
    
    # init setup on exp
    exp[i].setup(
        cut_sub_dfs[i], 
        target = 'rolling_mean_grouped_soil', 
        enforce_exogenous = False, 
        fold_strategy='sliding', 
        fh = 48, 
        session_id = 123, 
        fold = 3,
        use_gpu = False, # with GPU 3x longer times for compare_model() function
        ignore_features = ['orig_index', 'gradient']
        #numeric_imputation_exogenous = 'mean'
    )

In [ ]:
# show available models
for i in range(len(cut_sub_dfs)):
    print(exp[i].models())

You can use any of the two method i.e. Functional or OOP and even switch back and forth between two set of API's. The choice of method will not impact the results and has been tested for consistency.

## Check Stats
The `check_stats` function is used to get summary statistics and run statistical tests on the original data or model residuals.

In [ ]:
# check statistical tests on original data
for i in range(len(cut_sub_dfs)):
    print("This is the", i, "part of the data:")
    print(exp[i].check_stats())

## Compare Models

This function trains and evaluates the performance of all the estimators available in the model library using cross-validation. The output of this function is a scoring grid with average cross-validated scores. Metrics evaluated during CV can be accessed using the `get_metrics` function. Custom metrics can be added or removed using `add_metric` and `remove_metric` function.

In [ ]:
# compare baseline models
best = []
for i in range(len(cut_sub_dfs)):
    print("This is for the", i+1, "part of the dataset: ")
    best.append(exp[i].compare_models(
        n_select = 8, 
        fold = 3, 
        sort = 'R2',# 
        verbose = 1, 
        exclude = ['lar_cds_dt', 'lr_cds_dt', 'auto_arima', 'arima'],
        #include=['lr_cds_dt', 'br_cds_dt', 'ridge_cds_dt', 
        #         'huber_cds_dt', 'knn_cds_dt', 'catboost_cds_dt']
    ))

In [ ]:
for i in range(len(best)):
    print("\n The best model, for cut_sub_dfs[", i,"] is:")
    print(best[i][0])

Notice that the output between functional and OOP API is consistent. Rest of the functions in this notebook will only be shown using functional API only. 

___

## Analyze Model

You can use the `plot_model` function to analyzes the performance of a trained model on the test set. It may require re-training the model in certain cases.

In [ ]:
# plot forecast
for i in range(len(exp)):
    exp[i].plot_model(best[i], plot = 'forecast')

In [ ]:
# plot forecast for 36 months in future
for i in range(len(exp)):
    print("For the dataset:",i)
    exp[i].plot_model(best[i], plot = 'forecast', data_kwargs = {'fh' : 500})

In [ ]:
# plot forecast for 36 months in future
for j in range(len(best[0])):
    print("This is the ", j,  ". best model")
    for i in range(len(exp)):
        print("For the dataset:",i)
        exp[i].plot_model(best[i][j], plot = 'forecast', data_kwargs = {'fh' : 500})

In [ ]:
# residuals plot
for i in range(len(exp)):
    exp[i].plot_model(best[i], plot = 'residuals')

In [ ]:
# features plot
for i in range(len(exp)):
    exp[i].plot_model(best[i], plot = 'feature')

In [ ]:
# check docstring to see available plots 
#help(plot_model)

An alternate to `plot_model` function is `evaluate_model`. It can only be used in Notebook since it uses ipywidget.

## Prediction
The `predict_model` function returns `y_pred`. When data is `None` (default), it uses `fh` as defined during the `setup` function. 

In [ ]:
# predict on test set
holdout_pred = []
for i in range(len(exp)):
    holdout_pred.append(exp[i].predict_model(best[i][0]))

In [ ]:
# show predictions df
for i in range(len(holdout_pred)):
    print(holdout_pred[i].head(30))

In [ ]:
# generate forecast for 36 period in future
for i in range(len(exp)):
    print(exp[i].predict_model(best[i][0], fh = 36))

## Save Model

Finally, you can save the entire pipeline on disk for later use, using pycaret's `save_model` function.

In [ ]:
# save pipeline
for i in range(len(exp)):
    exp[i].save_model(best[i][0], 'my_weather_SPLIT_pipeline_TS_' + str(i))

In [ ]:
# load pipeline
loaded_best_pipeline = []
for i in range(len(exp)): # TODO: exp will not work if it is not set before, use OS later
    loaded_best_pipeline.append(load_model('my_weather_SPLIT_pipeline_TS_' + str(i)))
    loaded_best_pipeline

# 👇 Detailed function-by-function overview

## ✅ Setup
This function initializes the training environment and creates the transformation pipeline. Setup function must be called before executing any other function in PyCaret. `Setup` has only one required parameter i.e. `data`. All the other parameters are optional.

To access all the variables created by the setup function such as transformed dataset, random_state, etc. you can use `get_config` method.

In [ ]:
# check all available config
exp[0].get_config('memory')

In [ ]:
# lets access y_train_transformed
exp[0].get_config('y_train_transformed')

In [ ]:
# another example: let's access seed
#print("The current seed is: {}".format(exp[0].get_config('seed')))

# now lets change it using set_config
#set_config('seed', 786)
#print("The new seed is: {}".format(get_config('seed')))

All the preprocessing configurations and experiment settings/parameters are passed into the `setup` function. To see all available parameters, check the docstring:

In [ ]:
# help(setup)

## ✅  Check Stats
The `check_stats` function is used to get summary statistics and run statistical tests on the original data or model residuals.

In [ ]:
# check stats on original data, swap index of exp to see stats-> otherwise use print->ugly
exp[0].check_stats()

## ✅ Tune Model

The `tune_model` function tunes the hyperparameters of the model. The output of this function is a scoring grid with cross-validated scores by fold. The best model is selected based on the metric defined in optimize parameter. Metrics evaluated during cross-validation can be accessed using the `get_metrics` function. Custom metrics can be added or removed using `add_metric` and `remove_metric` function.

In [ ]:
# train a dt model with default params
#dt = create_model('dt_cds_dt')

best[0]

In [ ]:
best[0][6].__module__

In [ ]:
# tune hyperparameters of best without grid
def tune_all_without_grid(best):
    for j in range(len(exp)):
        tuned_best_models = []
        for i in range(len(best[j])):
            if re.search(r"^pycaret", best[j][i].__module__):
                print("This is for DataFrame", j, "This is for the",i,".best model:",best[j][i])
                tuned_best_models.append(exp[j].tune_model(best[j][i])) #TODO: Problem: not all estimators allow tuning without grid
            else:
                print("Cannot tune without grid:",best[j][i])
        tuned_best_models_array.append(tuned_best_models)
    return tuned_best_models_array
        
tuned_best_models_array = tune_all_without_grid(best)

Metric to optimize can be defined in `optimize` parameter (default = 'MASE'). Also, a custom tuned grid can be passed with `custom_grid` parameter. 

In [ ]:
for j in range(len(exp)):
    print("Dataset: ",j+1,"\n------------")
    for i in range(len(tuned_best_models_array[j])):
        print(i,". :",tuned_best_models_array[j][i])

In [ ]:
current = best[1][0]
print(str(current.regressor), type(str(current.regressor)))
print(current.get_params().keys())
print(type(current).__name__)
print(type(current).__name__.split('.')[-1])
exp[0].models(internal = True)

In [ ]:
for model in best[2]:
    column_reference = data.columns[0]
    print(model.__class__.__name__, column_reference)

In [ ]:
# define tuning grid -> no grid for model defined, TODO: come back later
#grid = {'regressor__max_depth' : [None, 2, 4, 6, 8, 10, 12]}
#sp = {'regressor__max_depth' : [None, 2, 4, 6, 8, 10, 12]}
# define the custom grid
param_grid_linear = {
    'degree': [1, 2, 3],
    'deseasonal_model': ['multiplicative', 'additive'],
    #'fe_target_rr': [True, False],
    #'regressor__copy_X': [True, False],
    #'regressor__fit_intercept': [True, False],
    'regressor__n_jobs': [-1, 1, 2],
    #'regressor__normalize': [True, False],
    'regressor__positive': [True, False],
    'sp': [1, 2, 3],
    'window_length': [10, 20, 30]
}

param_grid_knn = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': [10, 20, 30, 40, 50],
    'p': [1, 2]
}

params_grid = {
    'naive_bayes': {
        'alpha': [0.1, 0.5, 1.0],
        'fit_prior': [True, False]
    },
    'linear_regression': {
        'fit_intercept': [True, False],
        'normalize': [True, False]
    },
    'ridge': {
        'alpha': [0.1, 0.5, 1.0],
        'fit_intercept': [True, False],
        'normalize': [True, False],
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
    },
    'lasso': {
        'alpha': [0.1, 0.5, 1.0],
        'fit_intercept': [True, False],
        'normalize': [True, False],
        'selection': ['cyclic', 'random']
    },
    'elastic_net': {
        'alpha': [0.1, 0.5, 1.0],
        'l1_ratio': [0.1, 0.5, 0.9],
        'fit_intercept': [True, False],
        'normalize': [True, False],
        'selection': ['cyclic', 'random']
    },
    'knn': {
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance'],
        'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
        'leaf_size': [15, 20, 30, 40]
    },
    'decision_tree': {
        'criterion': ['gini', 'entropy'],
        'splitter': ['best', 'random'],
        'max_depth': [None, 5, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'random_forest': {
        'n_estimators': [10, 50, 100],
        'criterion': ['gini', 'entropy'],
        'max_depth': [None, 5, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'bootstrap': [True, False]
    },
    'gradient_boosting': {
        'n_estimators': [10, 50, 100],
        'learning_rate': [0.01, 0.1, 1],
        'max_depth': [3, 5, 10],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'subsample': [0.5, 0.7, 1],
        'max_features': ['auto', 'sqrt', 'log2']
    },
    'support_vector': {
        'C': [0.1, 0.5, 1.0],
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'degree': [2, 3, 4],
        'gamma': ['scale', 'auto']
    }
}

# tune model with custom grid and metric = MAE
tuned_best_models_grid_all = []
for i in range(len(exp)):
    print("This is for Dataset number: ",i+1)
    for j in range(len(best[i])):
        tuned_best_models_grid_exp = []
        print("This is for the",j+1,". best model:",best[i][0])
        for param in params_grid:
            if param ==
        
        
        
        if str(best[i][j].regressor) == "LinearRegression(n_jobs=-1)":
            tuned_best_models_grid_exp.append(exp[i].tune_model(best[i][j], custom_grid = param_grid_linear, optimize = 'MAE'))
        elif str(best[i][j].regressor) == "KNeighborsRegressor(n_jobs=-1)":
            tuned_best_models_grid_exp.append(exp[i].tune_model(best[i][j], custom_grid = param_grid_knn, optimize = 'MAE'))
    tuned_best_models_grid_all.append(tuned_best_models_grid_exp)

In [ ]:
# see tuned_dt params
for i in range(len(best_models)):
    print(tuned_best_models_grid[i])

In [ ]:
# to access the tuner object you can set return_tuner = True
tuners = []
tuned_best_models_tuner = []

for i in range(len(best_models)):
    tuned_model, tuner = tune_model(best_models[i], custom_grid = param_grid, return_tuner=True) #tuned_best_models_grid[i] does not work
    tuned_best_models_tuner.append(tuned_model)
    tuners.append(tuner)    

In [ ]:
# model object
for i in range(len(tuned_best_models_tuner)):
    tuned_best_models_tuner[i]

In [ ]:
# tuner object
for i in range(len(best_models)):
    tuner[i]

For more details on all available `search_library` and `search_algorithm` please check the docstring. Some other parameters that you might find very useful in `tune_model` are:

- choose_better
- custom_scorer
- n_iter
- search_algorithm
- optimize

You can check the docstring of the function for more info.

In [ ]:
# help(tune_model)

## ✅ Blend Models

In [ ]:
len(tuned_best_models_grid)

This function trains a `EnsembleForecaster` for select models passed in the estimator_list parameter. The output of this function is a scoring grid with CV scores by fold. Metrics evaluated during CV can be accessed using the `get_metrics` function. Custom metrics can be added or removed using `add_metric` and `remove_metric` function.

In [ ]:
# top 3 models based on mae
for i in range(len(best)):
    print(tuned_best_models_grid[i])

In [ ]:
# blend top 3 models
blended_model = []
for i in range(len(best_models)):
    print(tuned_best_models_grid[i])
    blended_model.append(blend_models(best_models))

Some other parameters that you might find very useful in `blend_models` are:

- choose_better
- method
- weights
- fit_kwargs
- optimize

You can check the docstring of the function for more info.

In [ ]:
# help(blend_models)

## ✅ Plot Model

This function analyzes the performance of a trained model on the hold-out set. It may require re-training the model in certain cases.

In [ ]:
blended_model[0][0]

In [ ]:
# extend forecast horizon

In [ ]:
# plot forecast
for i in range(len(best_models)):
    exp[i].plot_model(blended_model[i][0], plot = 'forecast', data_kwargs = {'fh' : 500}) #exp[i].plot_model(best[i], plot = 'forecast', data_kwargs = {'fh' : 500})

In [ ]:
future_df = exp[0].predict_model(best[0], fh=144)
print(future_df)

In [ ]:
cut_sub_dfs[0]['Time'][len(cut_sub_dfs[0])-1]

In [ ]:
cut_sub_dfs[0].tail()

In [ ]:
len(cut_sub_dfs)

In [ ]:
# TODO: for loop
all_dates = []
for i in range(len(cut_sub_dfs)):
    end = cut_sub_dfs[i]['Time'][len(cut_sub_dfs[i])-1]
    #print("end before adding: ", end)
    end=end+pd.Timedelta(days=2)
    #print("end after adding: ", end)
    all_dates.append(pd.date_range(start=cut_sub_dfs[i]['Time'][0], end=end, freq='10T')) #end= cut_sub_dfs[1]['Time'][len(cut_sub_dfs[1])-1]
    print("Dataframe",i,":",all_dates[i])

In [ ]:
# create new data (only dates)
new_data = []
for i in range(len(cut_sub_dfs)):
    new_data.append(pd.DataFrame())
    new_data[i]['Time'] = all_dates[i]
    new_data[i]['hour'] = [i.hour for i in new_data[i]['Time']]
    new_data[i]['minute'] = [i.minute for i in new_data[i]['Time']]
    new_data[i]['date'] = [i.day for i in new_data[i]['Time']]
    new_data[i]['month'] = [i.month for i in new_data[i]['Time']]
    
    print("Dataframe",i,":")
    print(new_data[i].head(1))

In [ ]:
new_data[0].head()

In [ ]:
future_df = []
for i in range(len(cut_sub_dfs)):
    future_df.append(exp[i].predict_model(best_models[i], X=new_data[i]))

In [ ]:
for i in range(len(cut_sub_dfs)):
    future_df[i].plot()

In [ ]:
# plot acf
# for certain plots you don't need a trained model
plot_model(plot = 'acf')

In [ ]:
# plot diagnostics
# for certain plots you don't need a trained model
plot_model(plot = 'diagnostics')

Some other parameters that you might find very useful in `plot_model` are:

- fig_kwargs
- data_kwargs
- display_format
- return_fig
- return_data
- save

You can check the docstring of the function for more info.

In [ ]:
# help(plot_model)

## ✅ Finalize Model
This function trains a given model on the entire dataset including the hold-out set.

In [ ]:
final_best = finalize_model(best) #TODO: for loop

In [ ]:
final_best

## ✅ Deploy Model
This function deploys the entire ML pipeline on the cloud.

**AWS:**  When deploying model on AWS S3, environment variables must be configured using the command-line interface. To configure AWS environment variables, type `aws configure` in terminal. The following information is required which can be generated using the Identity and Access Management (IAM) portal of your amazon console account:

- AWS Access Key ID
- AWS Secret Key Access
- Default Region Name (can be seen under Global settings on your AWS console)
- Default output format (must be left blank)

**GCP:** To deploy a model on Google Cloud Platform ('gcp'), the project must be created using the command-line or GCP console. Once the project is created, you must create a service account and download the service account key as a JSON file to set environment variables in your local environment. Learn more about it: https://cloud.google.com/docs/authentication/production

**Azure:** To deploy a model on Microsoft Azure ('azure'), environment variables for the connection string must be set in your local environment. Go to settings of storage account on Azure portal to access the connection string required.
AZURE_STORAGE_CONNECTION_STRING (required as environment variable)
Learn more about it: https://docs.microsoft.com/en-us/azure/storage/blobs/storage-quickstart-blobs-python?toc=%2Fpython%2Fazure%2FTOC.json

In [ ]:
# deploy model on aws s3
# deploy_model(best, model_name = 'my_first_platform_on_aws',
#             platform = 'aws', authentication = {'bucket' : 'pycaret-test'})

In [ ]:
# load model from aws s3
# loaded_from_aws = load_model(model_name = 'my_first_platform_on_aws', platform = 'aws',
#                              authentication = {'bucket' : 'pycaret-test'})

# loaded_from_aws

## ✅ Save / Load Model
This function saves the transformation pipeline and a trained model object into the current working directory as a pickle file for later use.

In [ ]:
# save model
for i in range(len(best_models)):
    save_model(best_models[i], 'my_first_SPLIT_model' + str(i))

In [ ]:
# load model
for i in range(len(best_models)): # will not work need to know how many models->use os TODO
    loaded_from_disk = load_model('my_first_SPILT_model'+str(i))
    loaded_from_disk

## ✅ Save / Load Experiment
This function saves all the experiment variables on disk, allowing to later resume without rerunning the setup function.

In [ ]:
# save experiment
save_experiment('my_SPLIT_experiment')

In [ ]:
# load experiment from disk
exp_from_disk = load_experiment('my_SPLIT_experiment', data=data)